<a href="https://colab.research.google.com/github/farkhanal/analisis-citra-wajah/blob/main/Training_Set_(ArcFace).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00


In [ ]:
# Import libraries yang diperlukan
import os
from mtcnn import MTCNN
from PIL import Image
import numpy as np

# Inisialisasi MTCNN face detector
detector = MTCNN()

# Path dataset dan path penyimpanan output
base_path = '/content/drive/My Drive/Colab Notebooks/nist_2/train/'
output_path = '/content/drive/My Drive/Colab Notebooks/nist_2/train/FaceDetection/'

# Membuat direktori FaceDetection jika belum ada
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Iterasi melalui setiap subfolder yang ada di base_path
for subfolder in os.listdir(base_path):
    subfolder_path = os.path.join(base_path, subfolder)

    # Memeriksa apakah path adalah sebuah direktori
    if os.path.isdir(subfolder_path):
        output_folder_path = os.path.join(output_path, subfolder)

        # Membuat subfolder FaceDetection per label jika belum ada
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)

        # Iterasi melalui setiap file dalam subfolder
        for file_name in os.listdir(subfolder_path):
            if file_name.endswith('.ppm'):  # Memeriksa format PPM
                file_path = os.path.join(subfolder_path, file_name)
                image = Image.open(file_path).convert('RGB')  # Membuka gambar

                # Konversi gambar ke array numpy untuk deteksi
                image_array = np.array(image)
                faces = detector.detect_faces(image_array)  # Deteksi wajah

                # Mengecek apakah ada wajah yang terdeteksi
                if faces:
                    # Ambil bounding box dari wajah pertama yang terdeteksi
                    x, y, width, height = faces[0]['box']
                    cropped_face = image.crop((x, y, x + width, y + height))

                    # Simpan gambar wajah yang telah dipotong
                    output_image_path = os.path.join(output_folder_path, file_name)
                    cropped_face.save(output_image_path)

                    # Tampilkan gambar yang telah dipotong
                    cropped_face.show()

print("Proses deteksi dan pemotongan wajah selesai!")

Proses deteksi dan pemotongan wajah selesai!


In [ ]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import cv2
import random

# Path dataset yang sudah dideteksi dan path penyimpanan hasil augmentasi
base_path = '/content/drive/My Drive/Colab Notebooks/nist_2/train/FaceDetection/'
output_path = '/content/drive/My Drive/Colab Notebooks/nist_2/AugmentedData/'

# Membuat direktori AugmentedData jika belum ada
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Fungsi untuk melakukan augmentasi
def augment_image(image):
    augmented_images = []

    # Rotasi
    angles = [-15, 15, -30, 30]
    for angle in angles:
        rotated = image.rotate(angle)
        augmented_images.append(rotated)

    # Flipping
    flipped_h = ImageOps.mirror(image)
    augmented_images.append(flipped_h)

    flipped_v = ImageOps.flip(image)
    augmented_images.append(flipped_v)

    # Pencahayaan dan Kontras
    enhancer = ImageEnhance.Brightness(image)
    bright_image = enhancer.enhance(1.5)
    dark_image = enhancer.enhance(0.5)
    augmented_images.append(bright_image)
    augmented_images.append(dark_image)

    enhancer_contrast = ImageEnhance.Contrast(image)
    high_contrast_image = enhancer_contrast.enhance(1.5)
    low_contrast_image = enhancer_contrast.enhance(0.5)
    augmented_images.append(high_contrast_image)
    augmented_images.append(low_contrast_image)

    # Gaussian Noise
    def add_gaussian_noise(image_np):
        mean = 0
        std = 10
        gauss = np.random.normal(mean, std, image_np.shape).astype('uint8')
        noisy = cv2.add(image_np, gauss)
        return Image.fromarray(noisy)

    image_np = np.array(image)
    augmented_images.append(add_gaussian_noise(image_np))

    # Blurring
    blurred = image.filter(ImageFilter.GaussianBlur(radius=2))
    augmented_images.append(blurred)

    # Warna atau Intensitas (Color Jitter)
    enhancer_color = ImageEnhance.Color(image)
    color_enhanced = enhancer_color.enhance(1.5)
    color_reduced = enhancer_color.enhance(0.5)
    augmented_images.append(color_enhanced)
    augmented_images.append(color_reduced)

    return augmented_images

# Iterasi melalui setiap subfolder di FaceDetection
for subfolder in os.listdir(base_path):
    subfolder_path = os.path.join(base_path, subfolder)
    output_folder_path = os.path.join(output_path, subfolder)

    # Membuat subfolder di AugmentedData jika belum ada
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Iterasi melalui setiap file dalam subfolder
    for file_name in os.listdir(subfolder_path):
        if file_name.endswith('.ppm'):
            file_path = os.path.join(subfolder_path, file_name)
            image = Image.open(file_path).convert('RGB')  # Membuka gambar

            # Lakukan augmentasi
            augmented_images = augment_image(image)

            # Simpan setiap gambar yang sudah diaugmentasi
            for i, aug_img in enumerate(augmented_images):
                aug_file_name = f"{file_name.split('.')[0]}_aug_{i}.ppm"
                aug_file_path = os.path.join(output_folder_path, aug_file_name)
                aug_img.save(aug_file_path)

print("Proses augmentasi selesai dan gambar tersimpan!")

Proses augmentasi selesai dan gambar tersimpan!


In [ ]:
import os
from PIL import Image

# Daftar direktori augmentasi yang akan digabungkan
augmentation_directories = [
    '/content/drive/My Drive/Colab Notebooks/nist_2/AugmentedData/',
    '/content/drive/My Drive/Colab Notebooks/nist_2/sample train_augment/'
]

# List untuk menyimpan data gambar dan label
images = []
labels = []

# Loop melalui setiap direktori augmentasi
for directory in augmentation_directories:
    for subfolder in os.listdir(directory):
        subfolder_path = os.path.join(directory, subfolder)
        if os.path.isdir(subfolder_path):  # Pastikan hanya membaca subfolder
            # Loop melalui setiap file gambar dalam subfolder
            for file_name in os.listdir(subfolder_path):
                if file_name.endswith('.ppm'):
                    file_path = os.path.join(subfolder_path, file_name)
                    # Baca gambar dan simpan label sesuai nama subfolder (misal: S001, S002, dst.)
                    image = Image.open(file_path).convert('RGB')
                    images.append(image)
                    labels.append(subfolder)  # Label sesuai dengan nama subfolder

# Menampilkan total gambar yang berhasil dimuat
print(f"Total gambar yang digabungkan dari augmentasi: {len(images)}")
print(f"Contoh label yang dimuat: {set(labels)}")

Total gambar yang digabungkan dari augmentasi: 7752
Contoh label yang dimuat: {'S185', 'S130', 'S013', 'S325', 'S383', 'S118', 'S153', 'S375', 'S321', 'S373', 'S133', 'S367', 'S272', 'S335', 'S336', 'S331', 'S324', 'S243', 'S058', 'S380', 'S117', 'S198', 'S138', 'S033', 'S108', 'S180', 'S389', 'S228', 'S427', 'S006', 'S405', 'S350', 'S195', 'S099', 'S258', 'S407', 'S227', 'S273', 'S390', 'S297', 'S387', 'S250', 'S428', 'S124', 'S073', 'S261', 'S226', 'S030', 'S346', 'S098', 'S044', 'S232', 'S036', 'S206', 'S174', 'S254', 'S259', 'S051', 'S392', 'S425', 'S398', 'S270', 'S283', 'S391', 'S385', 'S201', 'S052', 'S256', 'S381', 'S352', 'S429', 'S296', 'S364', 'S132', 'S222', 'S284', 'S396', 'S393', 'S354', 'S032', 'S031', 'S355', 'S341', 'S111', 'S288', 'S045', 'S307', 'S236', 'S096', 'S173', 'S394', 'S400', 'S409', 'S027', 'S239', 'S414', 'S310', 'S088', 'S202', 'S191', 'S412', 'S112', 'S093', 'S351', 'S253', 'S142', 'S223', 'S059', 'S220', 'S416', 'S008', 'S386', 'S134', 'S353', 'S417', '

In [ ]:
!pip install mtcnn opencv-python pillow matplotlib

In [ ]:
import os
from mtcnn import MTCNN
from PIL import Image
import numpy as np
import cv2

# Daftar direktori input untuk face alignment
input_directories = [
    '/content/drive/My Drive/Colab Notebooks/nist_2/AugmentedData/',
    '/content/drive/My Drive/Colab Notebooks/nist_2/sample train_augment/'
]
# Direktori output untuk hasil face alignment
output_directory = '/content/drive/My Drive/Colab Notebooks/nist_2/AlignedData/'

# Inisialisasi detektor MTCNN
detector = MTCNN()

# face alignment berdasarkan posisi mata
def align_face(image, keypoints):
    left_eye = keypoints['left_eye']
    right_eye = keypoints['right_eye']

    # Tentukan sudut antara mata kiri dan kanan
    delta_x = right_eye[0] - left_eye[0]
    delta_y = right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(delta_y, delta_x))

    # Hitung titik tengah antara kedua mata dan konversi ke float
    eyes_center = (float((left_eye[0] + right_eye[0]) / 2), float((left_eye[1] + right_eye[1]) / 2))

    # Dapatkan rotasi matriks untuk meluruskan gambar
    rot_mat = cv2.getRotationMatrix2D(eyes_center, angle, scale=1.0)

    # Lakukan rotasi gambar
    aligned_face = cv2.warpAffine(np.array(image), rot_mat, (image.width, image.height))

    return Image.fromarray(aligned_face)

# Proses face alignment untuk setiap direktori input
for directory in input_directories:
    for subfolder in os.listdir(directory):
        subfolder_path = os.path.join(directory, subfolder)
        if os.path.isdir(subfolder_path):
            # Buat subfolder untuk output face alignment jika belum ada
            output_subfolder = os.path.join(output_directory, subfolder)
            os.makedirs(output_subfolder, exist_ok=True)

            for file_name in os.listdir(subfolder_path):
                if file_name.endswith('.ppm'):
                    file_path = os.path.join(subfolder_path, file_name)
                    image = Image.open(file_path).convert('RGB')
                    image_array = np.array(image)

                    # Deteksi wajah dan landmark
                    result = detector.detect_faces(image_array)

                    if result:
                        # Ambil bounding box dan landmark wajah
                        keypoints = result[0]['keypoints']

                        # Face alignment menggunakan posisi mata
                        aligned_image = align_face(image, keypoints)

                        # Simpan hasil face alignment di direktori output
                        aligned_image.save(os.path.join(output_subfolder, file_name))

print("Face alignment selesai untuk semua direktori!")

Face alignment selesai untuk semua direktori!


In [ ]:
!pip install insightface onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055395 sha256=9b16be782b6930f6f3cb5444db617ab8895843c5a1ce17ddcb10f642197b7822
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface


In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import insightface
import pickle

# Direktori yang berisi hasil face alignment pada training set
aligned_data_dir = '/content/drive/My Drive/Colab Notebooks/nist_2/AlignedData/'

# Direktori untuk menyimpan basis data embedding
output_embedding_file = '/content/drive/My Drive/Colab Notebooks/nist_2/face_embeddings.pkl'

# Muat model ArcFace untuk ekstraksi fitur
model = insightface.app.FaceAnalysis(providers=['CPUExecutionProvider'])  # Ubah ke GPU jika ada
model.prepare(ctx_id=0, det_size=(640, 640))  # Sesuaikan dengan kebutuhan

# Dictionary untuk menyimpan embedding dan label
embeddings_dict = {}

# Loop melalui setiap subfolder di direktori aligned data
for subfolder in os.listdir(aligned_data_dir):
    subfolder_path = os.path.join(aligned_data_dir, subfolder)

    if os.path.isdir(subfolder_path):
        embeddings_dict[subfolder] = []  # Inisialisasi daftar untuk setiap subfolder (label)

        # Loop melalui setiap gambar di subfolder
        for file_name in tqdm(os.listdir(subfolder_path), desc=f"Processing {subfolder}"):
            if file_name.endswith('.ppm'):
                file_path = os.path.join(subfolder_path, file_name)

                # Buka gambar dan konversi ke format RGB
                image = Image.open(file_path).convert('RGB')
                image_np = np.array(image)

                # Ekstraksi fitur menggunakan ArcFace
                faces = model.get(image_np)

                # Proses hanya jika ada wajah yang terdeteksi
                if faces:
                    # Ambil embedding dari wajah pertama yang terdeteksi
                    embedding = faces[0].embedding

                    # Simpan embedding di dictionary sesuai dengan label subfoldernya
                    embeddings_dict[subfolder].append(embedding)

# Simpan embedding dan label ke file untuk basis data
with open(output_embedding_file, 'wb') as f:
    pickle.dump(embeddings_dict, f)

print("Ekstraksi fitur selesai dan embedding wajah disimpan sebagai basis data.")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:10<00:00, 27536.08KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Processing S420:   0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
Processing S001: 100%|██████████| 12/12 [00:14<00:00,  1.19s/it]

Ekstraksi fitur selesai dan embedding wajah disimpan sebagai basis data.


In [ ]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib  # Untuk menyimpan model

# Path ke file embedding yang sudah disimpan
embedding_file = '/content/drive/My Drive/Colab Notebooks/nist_2/face_embeddings.pkl'

# Path untuk menyimpan model k-NN yang sudah dilatih
model_save_path = '/content/drive/My Drive/Colab Notebooks/nist_2/knn_face_recognition_model.pkl'

# Load embedding dan label dari file
with open(embedding_file, 'rb') as f:
    embeddings_dict = pickle.load(f)

# Siapkan list untuk embedding dan label
X = []  # Embedding features
y = []  # Labels

# Pisahkan embedding dan label dari dictionary
for label, embeddings in embeddings_dict.items():
    for embedding in embeddings:
        X.append(embedding)
        y.append(label)

# Konversi ke array numpy untuk kompatibilitas dengan scikit-learn
X = np.array(X)
y = np.array(y)

# Split data menjadi train dan test set untuk evaluasi (opsional)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model k-NN, tentukan jumlah tetangga (k)
k = 5  # Anda bisa menyesuaikan k berdasarkan performa
knn_model = KNeighborsClassifier(n_neighbors=k, metric='cosine')

# Latih model dengan data train
knn_model.fit(X_train, y_train)
print("Model k-NN telah selesai dilatih.")

# Evaluasi model pada data test untuk melihat performa awal
y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi awal model k-NN pada data test: {accuracy * 100:.2f}%')

# Simpan model k-NN yang telah dilatih
joblib.dump(knn_model, model_save_path)
print(f"Model k-NN disimpan di {model_save_path}.")

Model k-NN telah selesai dilatih.
Akurasi awal model k-NN pada data test: 98.20%
Model k-NN disimpan di /content/drive/My Drive/Colab Notebooks/nist_2/knn_face_recognition_model.pkl.
